# Timescale Vector Store (PostgreSQL)


In [17]:
# import logging
# import sys

# Uncomment to see debug logs
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import SimpleDirectoryReader, StorageContext
from llama_index.indices.vector_store import VectorStoreIndex
from llama_index.vector_stores import TimescaleVectorStore
import textwrap
import openai

### Setup OpenAI
The first step is to configure the openai key. It will be used to created embeddings for the documents loaded into the index

In [18]:
# Get openAI api key by reading local .env file
# The .env file should contain a line starting with `OPENAI_API_KEY=sk-`
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# OR set it explicitly 
#import os
#os.environ["OPENAI_API_KEY"] = "<your key>"

openai.api_key = os.environ["OPENAI_API_KEY"]

### Loading documents
Load the documents stored in the `paul_graham_essay` using the SimpleDirectoryReader

In [19]:
documents = SimpleDirectoryReader("../data/paul_graham").load_data()
print("Document ID:", documents[0].doc_id)

Document ID: ecd1868c-7484-4aab-b7ba-6c70e0ae27df


### Get a Timescale service URL
You need a service url to connect to the Timescale instance.

To connect to your PostgreSQL database, you'll need your service URI, which can be found in the cheatsheet file you downloaded after creating a new database. The URI will look something like this: `postgres://tsdbadmin:<password>@<id>.tsdb.cloud.timescale.com:<port>/tsdb?sslmode=require`

In [20]:
# Get the service url by reading local .env file
# The .env file should contain a line starting with `TIMESCALE_SERVICE_URL=postgresql://`
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

TIMESCALE_SERVICE_URL = os.environ["TIMESCALE_SERVICE_URL"]

# OR set it explicitly
# TIMESCALE_SERVICE_URL = "postgres://tsdbadmin:<password>@<id>.tsdb.cloud.timescale.com:<port>/tsdb?sslmode=require"

### Create the index
Here we create an index backed by Timescale using the documents loaded previously. TimescaleVectorStore takes a few arguments.

In [ ]:
vector_store = TimescaleVectorStore.from_params(
    service_url=TIMESCALE_SERVICE_URL,
    table_name="paul_graham_essay",
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

### Query the index
We can now ask questions using our index.

In [22]:
query_engine = index.as_query_engine()
response = query_engine.query("Did the author work at YC?")

In [23]:
print(textwrap.fill(str(response), 100))

Yes, the author did work at YC.


In [24]:
response = query_engine.query("What did the author do growing up?")

In [25]:
print(textwrap.fill(str(response), 100))

The author worked on writing and programming growing up. They wrote short stories and tried writing
programs on an IBM 1401 computer. They also built a microcomputer and started programming on it,
writing simple games and a word processor.


### Querying existing index

In [ ]:
from llama_index import load_index_from_storage

vector_store = TimescaleVectorStore.from_params(
    service_url=TIMESCALE_SERVICE_URL,
    table_name="paul_graham_essay",
)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")

In [27]:
print(textwrap.fill(str(response), 100))

The author worked on writing and programming outside of school. They wrote short stories and tried
writing programs on an IBM 1401 computer. They also built a microcomputer and started programming on
it, writing simple games and a word processor.
